# **Prior force field for the deep learning CG Model of DNA**

This Jupyter notebook get the parameters for the prior force field from the full atomistic trajectories ....

---
---
# **Import of the necesary packages**

Firstly, we need to install all necessary libraries and packages for our mapping. The main packages are:

1.    Moleculekit
3.    Numpy (https://numpy.org/)

In [ ]:
import sys
import os
workDir = os.getcwd()
parentDir = os.path.abspath(os.path.join(workDir, os.pardir))
sys.path.append(os.path.join(parentDir, 'src'))
from src.prior_fit import get_param_bonded
from src.prior_fit import get_param_nonbonded
from src.prior_fit import get_param_nonbonded_rep
from src.make_deltaforces import make_deltaforces
import numpy as np
import yaml
from moleculekit.molecule import Molecule

In [ ]:
PDB_file = 'Data/DNA1_CG.pdb' 
DCD_file = 'Data/DNA1_CG.dcd'
mol = Molecule(PDB_file,validateElements=False)
mol.read(DCD_file)
mol.coords = mol.coords/10.0 # convert to nm
print(mol.coords.shape)
print(mol.name)


In [ ]:
# Define the CG topology
basepairs_number = 20
bonds = [[i, i + 1] for i in range(basepairs_number-1)]
bonds  += [[i + 20, i + 21] for i in range(basepairs_number-1)]
mol.bonds = np.array(bonds)
mol.bondtype = np.array(['un' for i in range(2*basepairs_number-2)])
angles = [[i, i + 1, i + 2] for i in range(basepairs_number-2)]
angles += [[i + 20, i + 21, i + 22] for i in range(basepairs_number-2)]
mol.angles = np.array(angles)

---
---
## **Prior dictionary**

The prior dictionary, that will form a force field file, need to be filled with fields: 

* `atomtypes` - stores unique bead names

* `bonds` - parameters describing bonded interactions. Both parameters will be calculated based on the training data.
    * `req` - equilibrium distance of the bond
    * `k0` - spring constant
    *  ```V = k * (x - x0)**2 + V0``` - Fitting function

* `lj` - parameters describing Lennard-Jones interactions. 
    * `epsilon` - will be calculated based on the training data.
    * `sigma` - in this case set to 1.0
    * ```V = 4*eps*((sigma/r)**12 - (sigma/r)**6) + V0``` - Fitting function

* `electrostatics` - parameters describing electrostatic interactions:
    * `charge` - in this case 

* `masses` - masses of the beads.

In [ ]:
# Initiate prior dictionary
priors = {}
priors['atomtypes'] = list(set(mol.atomtype))
priors['bonds'] = {}
priors['angles'] = {}
priors['morse'] = {}
priors['electrostatics'] = {at:{'charge': -1.0} for at in priors['atomtypes']}
priors['masses'] = {'DA': 1.0, 'DT': 1.0, 'DG': 1.0, 'DC': 1.0}

print(priors['atomtypes'])
print(priors['masses'])

### Bonded interactions

bonds approximated by harmonic function:

```V = k * (x - x0)**2 + V0```

angles approximated by cosine function:

```V = k * (1 - cosΘ) + V0```

In [ ]:
T = 298.15 # K
fit_range = [12.9,14.9] # Adjust this range to fit the model resolution

bond_params, angle_params = get_param_bonded(mol, fit_range, T)

priors['bonds'] = bond_params
priors['angles'] = angle_params

In [ ]:
print(priors['bonds'])
print(priors['angles'])

### Non-bonded interactions

non-bonded interactions approximated by a Lennard-Jones potential: 

```V = 4 * eps * ((sigma/r)**6) + V0```

In [ ]:
fit_range = {'A':[3.5,6.5], 'T':[3.5,6.5], 'G':[3.5,6.5], 'C':[3.5,6.5]}

nonbond_params = get_param_nonbonded_rep(mol, fit_range, T)

priors['morse'] = nonbond_params

In [ ]:
print(priors['morse'])

In [ ]:
with open("DNA1_priors.yaml","w") as f: 
    yaml.dump(priors, f)

---
---
## **Delta-Forces Preparation**
all the code inside **make_deltaforces.py** needs to be modified if we want to use delta-forces in the training

In [ ]:
forces = np.load('DNA1_CGfrc.npy')
delta_forces_npz = 'DNA1_deltaforces.npy'

make_deltaforces(mol, forces, delta_forces_npz, priors)